# LSTM Sample Implementation

In [130]:
import glob
import numpy as np
import tensorflow as tf

print('Tensorflow Version: {}'.format(tf.__version__))

Tensorflow Version: 1.8.0


## Data load

In [134]:
dgb = sorted(glob.glob('/home/miles/PycharmProjects/MultivariateTimeSeries/data/DGB_BTC/*'))

print('Number of samples: {}'.format(len(dgb)//2 ) )

Number of samples: 1973


In [26]:
features = np.load(dgb[0])
target = np.load(dgb[1])

In [65]:
features[np.newaxis, :, :].shape

(1, 28, 5)

In [67]:
# Drop `time` column and fill na (if any)
features = np.nan_to_num(np.delete(np.load(dgb[0]), 0, axis=1))[np.newaxis, :, :]
target = np.array(np.load(dgb[1])).reshape(1, -1)

print('Feature Dimensions: BATCHSIZE={}, TIMESTEPS={}, N_INPUTS={}'.format(*features.shape))
print('Target Dimensions: {}'.format(target.shape))

Feature Dimensions: BATCHSIZE=1, TIMESTEPS=28, N_INPUTS=5
Target Dimensions: (1, 1)


# Network Implementation

## Constants

In [101]:
TIME_STEPS = 28
N_INPUT = 5
N_HIDDEN = 128
LEARNING_RATE = 0.001
NUM_UNITS = 5

tf.reset_default_graph()

In [102]:
# Input placeholders
x = tf.placeholder(tf.float32, [None, TIME_STEPS, N_INPUT])
y = tf.placeholder(tf.float32, [None, 1])

In [107]:
out_weights = tf.Variable(tf.random_normal([NUM_UNITS, 1]))
out_bias = tf.Variable(tf.random_normal([1]))

In [103]:
# Reshape the input tensor from [batch_size, timesteps, n_input] to
# "timesteps" of [batch_size, n_input]
unstacked_inputs = tf.unstack(x, TIME_STEPS, 1)

In [104]:
lstm_layer = tf.contrib.rnn.BasicLSTMCell(NUM_UNITS, forget_bias=1)
outputs, _ = tf.contrib.rnn.static_rnn(lstm_layer, unstacked_inputs, dtype=tf.float32) 

In [108]:
predictions = tf.matmul(outputs[-1], out_weights) + out_bias

In [116]:
loss = tf.losses.mean_squared_error(y, predictions)
train_step = tf.train.AdamOptimizer(learning_rate=LEARNING_RATE).minimize(loss)

In [117]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(x, feed_dict={x: features})
    sess.run(y, feed_dict={y: target})
    sess.run(unstacked_inputs, feed_dict={x: features})
    _tmp = sess.run(predictions, feed_dict={x: features})
    
    sess.run(train_step, feed_dict={x: features, y: target})

In [122]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    feed_dict = {x: features, y: target}
    for step in range(500):
        sess.run(train_step, feed_dict=feed_dict)
        
        if step % 50 == 0:
            mse = sess.run(loss, feed_dict=feed_dict)
            print(f'Step={step}, Loss={mse}')
            
    p = sess.run(predictions, feed_dict=feed_dict)
        

Step=0, Loss=0.6586461663246155
Step=50, Loss=0.226825550198555
Step=100, Loss=2.144536210835213e-06
Step=150, Loss=1.3214857972343452e-05
Step=200, Loss=4.419606725036829e-08
Step=250, Loss=4.6569242906358e-10
Step=300, Loss=3.922697670161179e-15
Step=350, Loss=3.922697670161179e-15
Step=400, Loss=3.922697670161179e-15
Step=450, Loss=3.922697670161179e-15
